# Search AiiDA Database for Nanoribbons

In [ ]:
from aiida import load_dbenv, is_dbenv_loaded
from aiida.backends import settings
if not is_dbenv_loaded():
    load_dbenv(profile=settings.AIIDADB_PROFILE)
    
from aiida.orm.querybuilder import QueryBuilder
from aiida.orm.data.array.bands import BandsData
import nanoribbon

import ipywidgets as widgets
from IPython.display import display

In [ ]:
%%html

<!-- make widget labels wider -->
<style>
    .widget-label { min-width: 20ex !important; }
</style>

In [ ]:
def search(formula_list, expr, gap_range, homo_range, lumo_range, efermi_range, totmag_range, absmag_range, pk_list, progress):

        # html table header
        html  = '<style>#aiida_results td,th {padding: 2px}</style>' 
        html += '<form action="compare.ipynb" method="get" target="_blank">'
        html += '<table border=1 id="aiida_results" style="margin:10px;"><tr>'
        html += '<th></th>'
        html += '<th>PK</th>'
        html += '<th>Formula</th>'
        html += '<th>HOMO</th>'
        html += '<th>LUMO</th>'
        html += '<th>GAP</th>'
        html += '<th>Fermi Energy</th>'
        html += '<th>Total Mag.</th>'
        html += '<th>Abs Mag.</th>'
        html += '<th></th>'
        html += '</tr>'

        # query AiiDA database
        qb = QueryBuilder()
        qb.append(BandsData)

        # configure progress bar
        progress.max = qb.count()

        nmatch = 0
        for i, node_tuple in enumerate(qb.iterall()):
            node = node_tuple[0]
           
            # update progressbar, but not too often
            if i % 5 == 0:
                progress.value = i
            
            # filter PKs first (it's for free)
            if pk_list and str(node.pk) not in pk_list:
                continue

            props = nanoribbon.get_all_properties(node)
            
            # filter results
            if formula_list and props['formula'] not in formula_list:
                continue
            if expr and not eval(expr, props):
                continue
            if props['gap'] < gap_range[0] or props['gap'] > gap_range[1]:
                continue
            if props['homo'] < homo_range[0] or props['homo'] > homo_range[1]:
                continue
            if props['lumo'] < lumo_range[0] or props['lumo'] > lumo_range[1]:
                continue
            if props['fermi'] < efermi_range[0] or props['fermi'] > efermi_range[1]:
                continue
            if props['tmag'] < totmag_range[0] or props['tmag'] > totmag_range[1]:
                continue
            if props['amag'] < absmag_range[0] or props['amag'] > absmag_range[1]:
                continue

            nmatch += 1
            
            # append table row
            html += '<tr>'
            html += '<td><input type="checkbox" name="pk" value="%s"></td>'%node.pk
            html += '<td>%s</td>' % props['pk']
            html += '<td>%s</td>' % props['formula']
            html += '<td>%f</td>' % props['homo']
            html += '<td>%f</td>' % props['lumo']
            html += '<td>%f</td>' % props['gap']
            html += '<td>%f</td>' % props['fermi']
            html += '<td>%f</td>' % props['tmag']
            html += '<td>%f</td>' % props['amag']
            html += "<td><a target='_blank' href='./show.ipynb?pk=%s'>Show</a></td>"%node.pk
            html += '</tr>'

        html += '</table>'
        html += 'Found %d matches among %d entries.<br>'%(nmatch, qb.count())
        html += '<input type="submit" value="Compare">'
        html += '</form>'
        
        return html

In [ ]:
# search via criteria
layout = widgets.Layout(width="592px")
inp_pks = widgets.Text(description='PKs', placeholder='e.g. 4062 4753 (space separated)', layout=layout)
inp_formula = widgets.Text(description='Formulas:', placeholder='e.g. C44H16 C36H4', layout=layout)

def slider(desc, min, max):
    return widgets.FloatRangeSlider(description=desc, min=min, max=max, value=[min, max], step=0.05, layout=layout)    

inp_gap = slider("Gap:", 0.0, +3.0)
inp_homo = slider("HOMO:", -6.0, -3.0)
inp_lumo = slider("LUMO:", -5.0, -2.0)
inp_efermi = slider("Fermi Energy:", -6.0, -2.0)
inp_tmagn = slider("Total Magn.:", -6.0, +6.0)
inp_amagn = slider("Abs. Magn.:", 0.0, +20.0)
search_crit = [inp_pks, inp_formula, inp_gap, inp_homo, inp_lumo, inp_efermi, inp_tmagn, inp_amagn]

In [ ]:
# search via expression
expr_example_vars = {'pk':'123', 'formula':'CH66', 'homo':0.0, 'lumo':0.0, 
                     'gap':0.0, 'fermi':0.0, 'tmag':0.0, 'amag':0.0}

# # help text
# expr_help = "Write a valid boolean Python expression. Available variables are: <ul>"
# for k in expr_example_vars.keys():
#     expr_help += "<li>%s</li>"%k
# expr_help += "</ul>"

inp_expr = widgets.Text(description='Expression:', placeholder='e.g. fermi>-4 and "C44" in formula', layout=layout)
expr_valid = widgets.Valid(value=True)
expr_row = widgets.HBox([inp_expr, expr_valid])
search_crit.append(expr_row)

def on_expr_change(change):
    try:
        expr = inp_expr.value
        if(expr):
            bool(eval(expr, expr_example_vars))
        expr_valid.value = True
        expr_valid.description = ""
        button.disabled = False
    except Exception as e:
        expr_valid.value = False
        expr_valid.description = str(e)
        button.disabled = True

inp_expr.observe(on_expr_change, names='value')

In [ ]:
button = widgets.Button(description="Search")
results = widgets.HTML()
progress = widgets.IntProgress(description='Searching...')
progress.layout.visibility = "hidden"
app = widgets.VBox(children=search_crit + [button, progress, results])

In [ ]:
def on_click_search(b):
    # turn on busy indication
    results.value = ""
    button.layout.visibility = "hidden"
    progress.layout.visibility = "visible"
    inp_expr.disabled = True
    for w in search_crit:
        w.disabled = True
    
    #actual search
    results.value = search(pk_list=inp_pks.value.strip().split(),
                           formula_list=inp_formula.value,
                           expr=inp_expr.value,
                           gap_range=inp_gap.value,
                           homo_range=inp_homo.value,
                           lumo_range=inp_lumo.value,
                           efermi_range=inp_efermi.value,
                           totmag_range=inp_tmagn.value,
                           absmag_range=inp_amagn.value,
                           progress=progress)
    
    # turn off busy indication
    progress.layout.visibility = "hidden"
    button.layout.visibility = "visible"
    inp_expr.disabled = False
    for w in search_crit:
        w.disabled = False
    
# register event handler
button.on_click(on_click_search)

In [ ]:
display(app)